In [ ]:
import os, string
from labelbox_class import *
from utils import *
import numpy as np
from PIL import Image

In [ ]:
# Necessary inputs
api_key = ''
proj_id = ''

dataset_dir = '.\\..\\data\\datasets\\smaller_birds_dataset\\'

#'..\\data\\bird_lists\\bay_area_birds.txt'
birds_txt = '..\\data\\datasets\\smaller_birds_dataset\\short_bay_area_birds.txt'

raw_dir = dataset_dir + 'raw\\'
masks_dir = dataset_dir + 'masks\\'
training_dir = dataset_dir + 'training\\'

proj_name ='Short_Birds'
ndjson_file = dataset_dir + proj_name+ '.ndjson'

## !!! DANGER ZONE
clear_dirs = False

In [ ]:
## Clear dir
if clear_dirs:
    clear_dir(training_dir, [])
    clear_dir(masks_dir,[])

In [ ]:
# Finds all classes in the the dataset
birds = []
with open(birds_txt) as f:
    if f.readable() is False:
        raise FileNotFoundError("ERROR: File is not a readable file.")
    birds = f.readlines()
birds = [ string.capwords(x.strip()) for x in birds]
birds.sort()

In [ ]:
# Creates necessary directories in dataset directory
for bird in birds:
    if not os.path.exists(training_dir + bird + '\\'):
        os.makedirs(training_dir + bird + '\\')
    if not os.path.exists(masks_dir + bird + '\\'):
        os.makedirs(masks_dir + bird + '\\')

In [ ]:
# Makes a LabelBox Object from the project ID
labels = LabelBox(api_key=api_key, proj_id=proj_id)

# Exports the labels for the project that are marked as done to a ndjson file
labels.labelbox_export_labels(ndjson_file)

In [ ]:
# Create bounds dictionary from ndjson
print(f'{count_ndjson_lines(ndjson_file)} iterations')
bounds = labels.ndjson_get_bounds(ndjson_file)

In [ ]:
# Creates masks from bounds dictionary
masks = {key: create_mask(bounds[key]['polygon'], bounds[key]['width'], bounds[key]['height']) for key in bounds}

In [ ]:
# Create PNGs of masks, one with data written directly to it and another viewable one
for key in masks.keys():
    masks[key].save(os.path.join(masks_dir, key.split('__')[1].replace('.jpg',''), key.replace('.jpg','')+'.png'))